
#### Setting up the environment and import paths


In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...
remote: Enumerating objects: 4015, done.
remote: Counting objects: 100% (569/569), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 4015 (delta 340), reused 542 (delta 332), pack-reused 3446 (from 1)
Receiving objects: 100% (4015/4015), 49.69 MiB | 8.03 MiB/s, done.
Resolving deltas: 100% (2134/2134), done.


In [3]:
sys.path.insert(0,'boptestGymService')

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import requests
import random
import sys
import torch

from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper, DiscretizedActionWrapper
from stable_baselines3 import DQN

In [13]:
# Check if MPS is available
# NB! Will only work if you have a M1 Mac w/ a PyTorch version with MPS support. 
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [6]:
# for reproducibility
seed = 1337

In [7]:
url = 'https://api.boptest.net'

#### DQN init and train

In [14]:
env = BoptestGymEnv(
        url                  = url,
        testcase             = 'bestest_hydronic_heat_pump',
        actions              = ['oveHeaPumY_u'],
        observations         = {'time':(0,604800),
                                'reaTZon_y':(280.,310.),
                                'TDryBul':(265,303),
                                'HDirNor':(0,862),
                                'InternalGainsRad[1]':(0,219),
                                'PriceElectricPowerHighlyDynamic':(-0.4,0.4),
                                'LowerSetp[1]':(280.,310.),
                                'UpperSetp[1]':(280.,310.)},
        predictive_period    = 24*3600,
        regressive_period    = 6*3600,
        random_start_time    = True,
        max_episode_length   = 24*3600,
        warmup_period        = 24*3600,
        step_period          = 3600)

# Normalize observations and discretize action space
env = NormalizedObservationWrapper(env)
env = DiscretizedActionWrapper(env,n_bins_act=10)

# Instantiate an RL agent
model = DQN('MlpPolicy', env, device=device, verbose=1, gamma=0.99,
            learning_rate=5e-4, batch_size=24, seed=123456,
            buffer_size=365*24, learning_starts=24, train_freq=1)

# Main training loop
model.learn(total_timesteps=100)

# Loop for one episode of experience (one day as set in max_episode_length)
done = False
obs, _ = env.reset()
while not done:
  action, _ = model.predict(obs, deterministic=True)
  obs,reward,terminated,truncated,info = env.step(action)
  done = (terminated or truncated)

# Obtain KPIs for evaluation
env.get_kpis()

/Users/hanskrio/opt/miniconda3/envs/boptestgym/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Using mps device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 24       |
|    ep_rew_mean      | -7.39    |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 0        |
|    time_elapsed     | 379      |
|    total_timesteps  | 96       |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.294    |
|    n_updates        | 71       |
----------------------------------


{'tdis_tot': 37.15312778581881,
 'idis_tot': 0,
 'ener_tot': 0.4643638858761871,
 'cost_tot': 0.11771624506961344,
 'emis_tot': 0.07754876894132326,
 'pele_tot': 0.02370474504325742,
 'pgas_tot': None,
 'pdih_tot': None,
 'time_rat': 0.0010514094708142458}